In [1]:
# import all the important library
from tensorflow.keras.layers import Dense,Flatten,Lambda,Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import glob
import numpy as np
import matplotlib.pyplot as plt

In [2]:
image_size = [224,224]
train_path = '/content/drive/MyDrive/data/cats and dogs/train'
test_path= '/content/drive/MyDrive/data/cats and dogs/validation'

In [3]:
# import vgg16 models
vgg16 = VGG16(input_shape=image_size+[3],include_top=False,weights='imagenet')
for layer in vgg16.layers:
  layer.trainable = False

In [4]:
folder = glob.glob(train_path+'/*')

In [5]:
len(folder)

2

In [6]:
x = Flatten()(vgg16.output)
prediction = Dense(len(folder),activation='softmax')(x)
model = Model(inputs = vgg16.input,outputs = prediction)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
model.compile(loss ='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
## data augumentation
train_data_gen = ImageDataGenerator(rescale= 1./255,shear_range=0.2,
                                    zoom_range = 0.2,horizontal_flip = True)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [10]:
## training and test dataset
training_data = train_data_gen.flow_from_directory(directory=train_path,target_size=(224,224),
                                                   class_mode='categorical',batch_size = 32)
test_data =  train_data_gen.flow_from_directory(directory=test_path,target_size=(224,224),
                                                   class_mode='categorical',batch_size = 32)

Found 53 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [ ]:
#model_fitting
model_fit = model.fit_generator(training_data,validation_data=test_data,
                                epochs=100,steps_per_epoch = len(training_data),
                                validation_steps = len(test_data))

In [ ]:
## plotting loss,val_loss,accuracy and val_accuracy
plt.plot(model_fit.history['loss'],label = 'train_loss')
plt.plot(model_fit.history['val_loss'],label = 'test_loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(model_fit.history['accuracy'],label = 'train_accuracy')
plt.plot(model_fit.history['val_accuracy'],label = 'val_accuracy')
plt.legend()
plt.show()

In [ ]:
## saving model
from tensorflow.keras.models import load_model
model.save('model_vgg16.h5')

In [ ]:
#prediction
y_pred = model.predict(test_data)

In [ ]:
y_pred = np.argmax(y_pred,axis =1)


In [ ]:
##  making the single image prediction 
from tensorflow.keras.preprocessing.image import img_to_array,array_to_img,load_img
test_image = load_img('/content/drive/MyDrive/data/cats and dogs/validation/dog/dog.12202.jpg',target_size =(224,224))
test_image = img_to_array(test_image)
test_image = np.expand_dims(test_image,axis = 0)
test_image_prediction = model.predict(test_image)